In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer  # Added missing import
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("yds.csv")
# Quick overview of the data
print("Shape of dataset:", data.shape)
print(data.head())

In [ ]:
# Create a DataFrame with only rows where is_goal equals 1
df_goals = data[data['is_goal'] == 1].copy()

# Verify the result
print("Shape of DataFrame where is_goal == 1:", df_goals.shape)
print(df_goals.head())


In [ ]:
# 3. Check missing values (count & %)
missing_data = pd.DataFrame({
    'total_missing': data.isnull().sum(), 
    'perc_missing': (data.isnull().sum() / data.shape[0]) * 100
})
print(missing_data)

In [ ]:
# Filter rows where all three columns are missing
missing_all = data[data[['area_of_shot', 'shot_basics', 'range_of_shot']].isnull().all(axis=1)]

# Print the number of rows and, if needed, inspect the rows
print("Number of rows with all three columns missing:", len(missing_all))
print(missing_all)

# Remove rows where all three columns are missing
data = data[~data[['area_of_shot', 'shot_basics', 'range_of_shot']].isna().all(axis=1)]


In [ ]:
cols_to_check = ['area_of_shot', 'shot_basics', 'range_of_shot', 'is_goal']
data_complete = data.dropna(subset=cols_to_check, how='all')

print("Shape of dataset after removing rows where all these columns are missing:", data_complete.shape)


In [ ]:
print("Unique values in range_of_shot:", data['range_of_shot'].unique())

In [ ]:
# -----------------------------------------------
# 3. Football-Specific Feature Engineering 
# -----------------------------------------------
def create_football_features(df):
    df = df.copy()
    # Example: create a backcourt flag from range_of_shot (if applicable)
    df['is_backcourt'] = df['range_of_shot'] == 'Back Court Shot'
    # Optionally enforce domain rules (e.g., if backcourt then force a certain area/shot type)
    df.loc[df['is_backcourt'], 'area_of_shot'] = 'Mid Ground (MG)'
    df.loc[df['is_backcourt'], 'shot_basics'] = df.loc[df['is_backcourt'], 'shot_basics']\
        .where(df['shot_basics'].isin(['Mid Ground Line', 'Penalty Spot']), 'Mid Ground Line')
    return df

data = create_football_features(data)

# -----------------------------------------------
# 4. Define Evaluation and Imputation Functions
# -----------------------------------------------
def evaluate_imputation_model(df, target_col, predictor_cols, model, random_state=42, mask_fraction=0.1):
    """
    Evaluate a model for imputing a categorical target column by masking a fraction
    of complete data and computing accuracy.
    """
    df_complete = df[df[target_col].notnull()].copy()
    if df_complete.empty:
        print(f"No complete data available for {target_col}")
        return None
    
    le_target = LabelEncoder()
    df_complete[target_col + '_enc'] = le_target.fit_transform(df_complete[target_col])
    
    for col in predictor_cols:
        le = LabelEncoder()
        df_complete[col] = df_complete[col].astype(str)
        df_complete[col + '_enc'] = le.fit_transform(df_complete[col])
    
    features = [col + '_enc' for col in predictor_cols]
    X = df_complete[features]
    y = df_complete[target_col + '_enc']
    
    np.random.seed(random_state)
    mask = np.random.rand(len(df_complete)) < mask_fraction
    y_true = y[mask].copy()
    
    model.fit(X, y)
    y_pred = model.predict(X[mask])
    
    acc = accuracy_score(y_true, y_pred)
    print(f"Model evaluation for {target_col} using {model.__class__.__name__}: Accuracy = {acc:.4f}")
    return acc

def impute_categorical_model(df, target_col, predictor_cols, model, random_state=42):
    """
    Impute missing values in a target categorical column using a predictive model.
    This version handles previously unseen labels in predictor columns by replacing them
    with the mode from the training set.
    """
    # Separate rows where target is present (for training) and where it's missing (for prediction)
    train_df = df[df[target_col].notnull()].copy()
    test_df = df[df[target_col].isnull()].copy()
    
    if test_df.empty:
        print(f"No missing values in {target_col} to impute using {model.__class__.__name__}.")
        return df
    
    # Encode the target column using LabelEncoder on training data
    le_target = LabelEncoder()
    train_df[target_col + '_enc'] = le_target.fit_transform(train_df[target_col])
    
    # For each predictor, encode training data and then process test data:
    for col in predictor_cols:
        le = LabelEncoder()
        # Convert column to string (if not already)
        train_df[col] = train_df[col].astype(str)
        test_df[col] = test_df[col].astype(str)
        # Fit on training data
        train_df[col + '_enc'] = le.fit_transform(train_df[col])
        
        # In test data, replace unseen values with the mode from training data
        unseen_mask = ~test_df[col].isin(le.classes_)
        if unseen_mask.any():
            mode_val = train_df[col].mode()[0]
            test_df.loc[unseen_mask, col] = mode_val
        
        # Transform test data
        test_df[col + '_enc'] = le.transform(test_df[col])
    
    # Prepare training features and target
    X_train = train_df[[col + '_enc' for col in predictor_cols]]
    y_train = train_df[target_col + '_enc']
    
    # Fit the chosen model on the training data
    model.fit(X_train, y_train)
    
    # Prepare test features and predict missing target values
    X_test = test_df[[col + '_enc' for col in predictor_cols]]
    y_pred_enc = model.predict(X_test)
    y_pred = le_target.inverse_transform(y_pred_enc)
    
    # Fill the missing values in the original DataFrame
    df.loc[df[target_col].isnull(), target_col] = y_pred
    print(f"Imputed missing values in '{target_col}' using {model.__class__.__name__}.")
    return df


# -----------------------------------------------
# 5. Set Up Target and Predictor Columns
# -----------------------------------------------
# For shot_basics and range_of_shot, we use predictors: area_of_shot and range_of_shot or shot_basics respectively.
# For area_of_shot, we now include additional predictors 'location_x' and 'location_y' to improve accuracy.
target_columns = ['area_of_shot', 'shot_basics', 'range_of_shot']
predictor_columns = {
    'area_of_shot': ['shot_basics', 'range_of_shot', 'location_x', 'location_y'],  # extra predictors added
    'shot_basics': ['area_of_shot', 'range_of_shot'],
    'range_of_shot': ['area_of_shot', 'shot_basics']
}

# -----------------------------------------------
# 6. Evaluate Imputation Models for Each Target Column
# -----------------------------------------------
evaluation_results = {}
# Initialize models
dt_model = DecisionTreeClassifier(random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
et_model = ExtraTreesClassifier(n_estimators=100, random_state=42)

print("\nEvaluating imputation for target columns...\n")
for target in target_columns:
    preds = predictor_columns[target]
    if target == 'area_of_shot':
        # Evaluate three models for area_of_shot
        acc_dt = evaluate_imputation_model(data, target, preds, dt_model)
        acc_rf = evaluate_imputation_model(data, target, preds, rf_model)
        acc_et = evaluate_imputation_model(data, target, preds, et_model)
        best_model_name = None
        best_acc = max(acc_dt, acc_rf, acc_et)
        if best_acc == acc_et:
            best_model_name = "ExtraTrees"
        elif best_acc == acc_rf:
            best_model_name = "RandomForest"
        else:
            best_model_name = "DecisionTree"
        evaluation_results[target] = {"DecisionTree": acc_dt, "RandomForest": acc_rf, "ExtraTrees": acc_et, "Best": best_model_name}
    else:
        # Evaluate for shot_basics and range_of_shot using DT and RF
        acc_dt = evaluate_imputation_model(data, target, preds, dt_model)
        acc_rf = evaluate_imputation_model(data, target, preds, rf_model)
        best_model_name = "DecisionTree" if acc_dt >= acc_rf else "RandomForest"
        evaluation_results[target] = {"DecisionTree": acc_dt, "RandomForest": acc_rf, "Best": best_model_name}

print("\nEvaluation Results:")
print(evaluation_results)

# -----------------------------------------------
# 7. Impute Missing Values Using the Best Model for Each Target Column
# -----------------------------------------------
final_imputed_data = data.copy()
for target in target_columns:
    preds = predictor_columns[target]
    if target == 'area_of_shot':
        if evaluation_results[target]["Best"] == "ExtraTrees":
            final_imputed_data = impute_categorical_model(final_imputed_data, target, preds, et_model)
        elif evaluation_results[target]["Best"] == "RandomForest":
            final_imputed_data = impute_categorical_model(final_imputed_data, target, preds, rf_model)
        else:
            final_imputed_data = impute_categorical_model(final_imputed_data, target, preds, dt_model)
    else:
        if evaluation_results[target]["Best"] == "DecisionTree":
            final_imputed_data = impute_categorical_model(final_imputed_data, target, preds, dt_model)
        else:
            final_imputed_data = impute_categorical_model(final_imputed_data, target, preds, rf_model)

# -----------------------------------------------
# 8. Post-Imputation Validation
# -----------------------------------------------
print("\nMissing values after model-based imputation in target columns:")
print(final_imputed_data[target_columns].isnull().sum())

# For error analysis, we can optionally generate a classification report

def error_analysis(target_col):
    """
    Generate classification report for the target column using a pipeline.
    If any class has fewer than 6 samples, the SMOTE step is skipped.
    Uses RandomForestClassifier instead of CatBoostClassifier.
    """
    predictors = predictor_columns[target_col][:2]
    df_complete = final_imputed_data.dropna(subset=[target_col]).copy()
    X = df_complete[predictors]
    y = df_complete[target_col]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42)
    
    # Build pipeline steps conditionally
    steps = [
        ('encoder', ColumnTransformer([
            ('enc', OneHotEncoder(handle_unknown='ignore'), predictors)
        ], remainder='passthrough'))
    ]
    
    # Check class sizes to determine if SMOTE is feasible
    class_counts = y_train.value_counts()
    if class_counts.min() < 6:
        print(f"Some classes in {target_col} have fewer than 6 samples; skipping SMOTE.")
    else:
        steps.append(('smote', SMOTE(random_state=42)))
    
    steps.append(('classifier', RandomForestClassifier(n_estimators=100, random_state=42)))
    
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    print(f"\n{target_col} Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Weighted F1 Score: {f1:.4f}\n")
# Now run error analysis for each target column
for col in ['area_of_shot', 'shot_basics', 'range_of_shot']:
    error_analysis(col)

In [ ]:
#remaining min,sec
cols_to_plot = ['remaining_min', 'remaining_min.1', 'remaining_sec', 'remaining_sec.1']
existing_cols = [col for col in cols_to_plot if col in data.columns]

for col in existing_cols:
    plt.figure(figsize=(6,4))
    sns.boxplot(y=data[col])
    plt.title(f"Boxplot of {col}")
    plt.ylabel(col)
    plt.show()

In [ ]:
# Due to prsence of high number of outliers in remianing.min.1 and remaining.sec.1 whose values are skewed which could affect analysis from precision and accuracy POV,
# we remove these columns
# Define the duplicate columns to remove
cols_to_drop = ["remaining_min.1","remaining_sec.1","power_of_shot","power_of_shot.1","knockout_match.1"]

# Remove these columns from the DataFrame
data.drop(cols_to_drop, axis=1, inplace=True)

# Optionally, check the remaining columns
print("Columns after removal:")
print(data.columns.tolist())



In [ ]:
# Remove the 'is_backcourt' column from the DataFrame 'data'
data.drop('is_backcourt', axis=1, inplace=True)

# Verify that the column has been removed
print("Columns after removal:")
print(data.columns.tolist())

In [ ]:
data.drop('distance_of_shot.1',axis=1,inplace=True)
# Verify that the column has been removed
print("Columns after removal:")
print(data.columns.tolist())

In [ ]:
df_copy = data.copy()

In [ ]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas


In [ ]:
from snowflake.snowpark import Session

# 1. Obtain the active session in your Snowflake Notebook
session = Session.get_active_session()

# 2. Suppose 'df_copy' is your final Pandas DataFrame
# Convert it to a Snowpark DataFrame
snowpark_df = session.create_dataframe(df_copy)



In [ ]:
# 1. Create or replace a temporary stage
session.sql("CREATE OR REPLACE STAGE my_temp_stage").collect()

# 2. Write the Snowpark DataFrame to the stage as a CSV file
snowpark_df.write.copy_into_location(
    location="@my_temp_stage/mydata.csv",
    file_format_type="csv",
    header=True,
    overwrite=True
)

# 3. Download the file from the stage to your local machine
session.file.get("@my_temp_stage/mydata.csv", "mydata.csv")

print("File downloaded as 'mydata.csv'")


In [ ]:
# Download the file from the stage to the /tmp folder
session.file.get("@my_temp_stage/mydata.csv", "/tmp/mydata.csv")
print("File downloaded from stage to /tmp/mydata.csv")



In [ ]:
import os

# List all files in the /tmp folder
files_in_tmp = os.listdir('/tmp')
print(files_in_tmp)

In [ ]:
from IPython.display import FileLink, display

# Create a clickable download link for the specific file
download_link = FileLink(r'/tmp/mydata.csv/mydata.csv_0_0_0.csv.gz')
display(download_link)
